# Mapping Agricultural-Driven Deforestation in Central Kalimantan with Remote Sensing

Lu Yii Wong 

May 2025

## Part 1: Setup and Data Aquisition 

In [1]:
# Setup Libraries
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd
from datetime import datetime

In [2]:
# Authenticate GEE
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')

In [7]:
# Pull Sentinel-2 Imagary using GEE

# Load Boundary for Area of Interest (Pulang Pisau)
gdf = gpd.read_file("data/Pulang_Pisau_boundary.geojson").to_crs(epsg=4326)
roi = geemap.geopandas_to_ee(gdf)

# Load Sentinel-2 SF Image (2A)
sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR")
             .filterDate("2018-12-01", "2019-12-31")
             .filterBounds(roi)
             .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30)))  # Optional: <30% cloud cover

# Function to mask clouds and cirrus using QA60 bitmask
def maskS2clouds(image):
    qa = image.select("QA60")
    cloudBitMask = 1 << 10  # Bit 10: clouds
    cirrusBitMask = 1 << 11 # Bit 11: cirrus
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])

In [8]:
# Apply cloud masking 
sentinel2_clean = sentinel2.map(maskS2clouds)

# Select bands 
sentinel2_selected = sentinel2_clean.select(["B8", "B12", "B4", "B11", "B3", "B2", "B5", "B6", "B7", "B8A"])

In [9]:
# Create Visual Parameters and Display map
viz_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0.0,
    "max": 0.3
}

Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(sentinel2_selected.median().clip(roi), viz_params, "Sentinel-2 Median (2018-2019)")
Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map

Map(center=[-2.7158640396563944, 113.95461205914583], controls=(WidgetControl(options=['position', 'transparen…

## Calculate Normalized Burn Ratio (NBR) 

In [10]:
# Create a function to calculate NBR for each Sentinel-2 image
def calculate_nbr(image):
    # NBR = (NIR - SWIR2) / (NIR + SWIR2)
    nbr = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    return image.addBands(nbr)

# Apply the NBR function across the cleaned Sentinel-2 collection
sentinel2_with_nbr = sentinel2_selected.map(calculate_nbr)

# Only keep the NBR band for easier processing later
nbr_collection = sentinel2_with_nbr.select("NBR")

# Visualize the median NBR across the whole period
Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(nbr_collection.median().clip(roi), {"min": -0.5, "max": 1, "palette": ["white", "green", "black"]}, "Median NBR (2018-2019)")
Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map

Map(center=[-2.7158640396563944, 113.95461205914583], controls=(WidgetControl(options=['position', 'transparen…

In [11]:
# Check number of images in the filtered Sentinel-2 ImageCollection
n_images = nbr_collection.size().getInfo()
print(f"Number of Sentinel-2 images with NBR in the collection: {n_images}")

Number of Sentinel-2 images with NBR in the collection: 234


In [14]:
# Get list of image dates
timestamps = nbr_collection.aggregate_array("system:time_start").getInfo()
dates = [datetime.utcfromtimestamp(t / 1000).strftime('%Y-%m-%d') for t in timestamps]

# Create a DataFrame
dates_df = pd.DataFrame(dates, columns=["Acquisition Date"])
dates_df.index += 1
dates_df


,Acquisition Date
1,2018-12-30
2,2018-12-30
3,2018-12-30
4,2019-02-08
5,2019-02-08
...,...
230,2019-12-05
231,2019-12-05
232,2019-12-20
233,2019-12-20
